In [1]:
import matplotlib.pyplot as plt

from tqdm import tqdm

from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval

#from utils_func import corpus_processing, matrix_creation, clustering, retriever_model

import os

/home/maxkev/anaconda3/lib/python3.12/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import spacy
from tqdm import tqdm
import pandas as pd

from multiprocessing import Pool, cpu_count
import multiprocessing
multiprocessing.set_start_method("spawn", force=True)

nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words
clean_tokens = lambda tokens : ' '.join([token.lemma_.lower() for token in tokens if token.lemma_.lower() not in stopwords and not token.is_punct])

def pre_process(elem_to_preprocess: tuple[int, dict[str,str]]) -> tuple[int, str]:
  """
  Preprocesses the text data in the corpus\n
  :param elem_to_preprocess: Tuple containing the key and the value of the element to preprocess\n
  :return: Tuple containing the key and the preprocessed text data\n
  """

  key, val = elem_to_preprocess
  return key, f"{clean_tokens(nlp(val['title'].lower()))} {clean_tokens(nlp(val['text'].lower()))}" # Cleaning the text document

# Helper function to process a single key-value pair
def process_item(item):
    key, value = item
    return key, pre_process((key, value))[1]

def preprocess_corpus_dict(corpus: dict[int, dict[str, str]]) -> dict[int, str]:
    """
    Preprocesses the text data in the corpus in parallel\n
    :param corpus: The corpus to preprocess
    :return: The preprocessed corpus
    """
    # Convert the corpus to a list of items for parallel processing
    items = list(corpus.items())

    # Use a multiprocessing pool for parallel execution
    with Pool(cpu_count()) as pool:
        results = list(tqdm(pool.imap(process_item, items), total=len(items)))

    # Combine results into a dictionary
    cleaned_corpus = {key: value for key, value in results}

    return cleaned_corpus

""" Non parallel version

def preprocess_corpus_dict(corpus: dict[int, dict[str,str]]) -> dict[int, str]:
    '''
    Preprocesses the text data in the corpus\n
    :param corpus: The corpus to preprocess\n
    :return: The preprocessed corpus\n
    '''
    cleaned_corpus = {}
    for key in tqdm(corpus.keys()):
        cleaned_corpus[key] = pre_process((key, corpus[key]))[1]
    return cleaned_corpus
"""

def get_unique_words(corpus:dict[int, str]) -> set:
    """
    Function to find the unique words in a corpus
    :param corpus: dict[int, str] - a dictionary with the key being the document id and the value being the document text
    :return: set - a set of unique words in the corpus
    """
    unique_words = set()
    for doc_id in tqdm(corpus, desc="Getting unique words"):
        text = corpus[doc_id]
        text = text.split()
        for word in text:
            unique_words.add(word)
    return unique_words

def save_processed_corpus(corpus, path_to_save):
    """
    Saves the preprocessed corpus to a csv file\n
    :param corpus: The preprocessed corpus\n
    :param path_to_save: The path to save the preprocessed corpus\n
    """

    df = pd.DataFrame.from_dict(corpus, orient='index')
    df.reset_index(inplace=True)
    df.columns = ["doc_id", "text"]
    df.to_csv(path_to_save, index=False)

In [3]:
from utils_func import corpus_processing

In [4]:
# Example corpus and queries (replace with your actual data)
dataset = "nfcorpus"
#dataset = "fiqa"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
data_path = f"datasets/{dataset}"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/3633 [00:00<?, ?it/s]

In [5]:
cleaned_corpus = corpus_processing.preprocess_corpus_dict(corpus)

Prétraitement du corpus: 100%|██████████| 3633/3633 [00:18<00:00, 197.50it/s]


In [6]:
import tempfile

with tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8') as temp_file:
    for keys in tqdm(cleaned_corpus.keys()):
        temp_file.write(f"{cleaned_corpus[keys]}\n")
    temp_file_path = temp_file.name

100%|██████████| 3633/3633 [00:00<00:00, 367249.26it/s]


In [7]:
import tempfile
import fasttext
import re
import numpy as np

'''
with tempfile.NamedTemporaryFile(delete=False, mode='w', encoding='utf-8') as temp_file:
    temp_file.write(text)
    temp_file_path = temp_file.name
'''
# Entraîner le modèle FastText avec le fichier temporaire
model = fasttext.train_unsupervised(temp_file_path, model='skipgram')
# Liste des mots à visualiser (tu peux la personnaliser)

#words = re.findall(r'\b\w+\b', text.lower())
unique_words = list(get_unique_words(cleaned_corpus))
#unique_words = list(set(words))

# Obtenir les vecteurs pour les mots choisis
word_vectors = np.array([model.get_word_vector(word) for word in unique_words])

embeddings = pd.DataFrame(word_vectors, index=unique_words)
embeddings.to_csv(f"word_vectors_{dataset}.csv", sep = ' ')
'''
# Path to the output file
output_file = f"word_vectors_{dataset}.txt"

# Open the file in write mode
with open(output_file, "w") as f:
    # Loop over each word and its corresponding vector
    for word, vector in zip(unique_words, word_vectors):
        # Convert the vector to a space-separated string
        vector_str = ' '.join(map(str, vector))
        # Write the word and its vector, followed by a newline
        f.write(f"{word} {vector_str}\n")
'''

model.save_model(f'fasttext_{dataset}')

Read 0M words
Number of words:  8630
Number of labels: 0
Progress: 100.0% words/sec/thread:   32343 lr:  0.000000 avg.loss:  2.150900 ETA:   0h 0m 0s
Getting unique words: 100%|██████████| 3633/3633 [00:00<00:00, 60165.15it/s]


In [8]:
len(unique_words)

303557